<img src="../../images/NxLogoTransparent.png" alt="Nx Icon" width=200px align=right /> 


# Workshop: Transformaciones de Dataframes con Snowpark

### Escenario: 

En este ejercicio, usarás la API Snowpark para examinar un conjunto de datos sin procesar y transformarlo en un DataFrame, al cual los analistas de su organización puedan explorar fácilmente.

### Pasos:
1. Instalar las librerías y configurar los parámetros de conexión.
2. Conectarnos y crear un objeto Sesion.
3. Examinar los datasets.
4. Explorar los datasets y ejecutar transformaciones.
5. (Anexo) Usar una UDF para realizar transformaciones.

### 1. Instalar las librerías y configurar los parámetros de conexión.


Inicialice una variable config, los parámetros necesasrios para conectarse a su cuenta de Snowflakea.
> **&#128221; Nota:** Actualice los valores con los datos de su cuenta free trial

In [ ]:
%pip install ipython-sql

In [ ]:
import os
import getpass
from urllib.parse import quote

# Load Jupyter/IPython sql magic
%load_ext sql

In [ ]:
# Gather account credentials
sf_account   = input('Snowflake Account: ') #qz79006.us-east-2.aws
sf_user      = input('Snowflake User: ') #WORKSHOP_USER
sf_password      = input('Snowflake Password: ') #WORKSHOP_USER_PASSWORD

# Generate default object names
wh_name    = f"COMPUTE_WH"
db_name    = f"SNOWPARK_DEMO_DB"

print("\r\nAccount credentials gathered. Select the next code cell to continue.")

### 2. Conectarnos y crear un objeto Sesion.

El siguiente bloque de código, creará un objeto de tipo `Session` y se conectará a tu cuenta de Snowflake.

*No es necesario modificar nada. Solamente ejecuta el bloque de código.*
> &#10071; Para Windows se requiere que se actualice el path del archivo `spp_utils_python.ipynb`.

In [ ]:
# Run utils notebook
%run ../utils/spp_utils_python.ipynb

connection_parameters = {
    "account": sf_account.upper(),
    "user": sf_user.upper(),
    "password": sf_password
}  

session = Session.builder.configs(connection_parameters).create()

### 3. Examinar los Datasets.

Importe las librerías `functions` y `types` para usarlas durante el workshop.

Cree dos `DataFrame` a partir de las tablas `campaign_spend` y `monthly_revenue`, luego examinelas para descubrir qué tipo de información está disponible en estos Datasets.

*No necesitas editar nada en la siguiente celda. Simplemente ejecútalo.*

In [ ]:
# Import Snowpark functions and types
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# Create a DataFrame for the table 
snow_df_campaing_spend = session.table('SNOWPARK_DEMO_DB.SNOWPARK_DEMO_SCHEMA.CAMPAIGN_SPEND')
snow_df_revenue = session.table('SNOWPARK_DEMO_DB.SNOWPARK_DEMO_SCHEMA.MONTHLY_REVENUE')

print('\nTable CAMPAIGN_SPEND')
for field in snow_df_campaing_spend.schema.fields:
    print(field)

print('\nTable MONTHLY_REVENUE')
for field in snow_df_revenue.schema.fields:
    print(field)

### 4. Explorar los datasets y ejecutar transformaciones.

#### 4.1 Seleccione columnas del DataFrame snow_df_campaing_spend.

Seleccione los siguientes campos del DataFrame `snow_df_campaing_spend` usando el método `select()` (recuerde encerrar los nombres de las columnas en una llamada al método `col()`):
- CAMPAIGN
- CHANNEL
- DATE
- TOTAL_CLICKS
- TOTAL_COST
- ADS_SERVED


*TIP: ¿Quieres echar un vistazo a los resultados de tus transformaciones? Utilice la* función `show()` *para ejecutar la consulta y mostrar las primeras diez filas.*

> **&#128221; Nota:** The [Snowpark API Reference (Python)](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/index.html#snowpark-api-reference-python)


In [ ]:
snow_df_spend = (snow_df_campaing_spend
        .select(
             col("CAMPAIGN")
            ,col("CHANNEL")
            ,col("DATE")
            ,col("TOTAL_CLICKS")
            ,col("TOTAL_COST")
            ,col("ADS_SERVED")
          ))

# Uncomment the following 'action' statement to execute transformations and view first ten results
#snow_df_spend.show()

#### 4.2 Gasto total por año y mes para todos los canales.

Transformemos los datos para que podamos ver el costo total por año/mes por canal usando las funciones `group_by()` y `agg()` Snowpark DataFrame.

In [ ]:
from snowflake.snowpark.functions import month,year,col,sum

snow_df_spend_per_channel = snow_df_spend.group_by(year('DATE'), month('DATE'),'CHANNEL').agg(sum('TOTAL_COST').as_('TOTAL_COST')).with_column_renamed('"YEAR(DATE)"',"YEAR").with_column_renamed('"MONTH(DATE)"',"MONTH").sort('YEAR','MONTH')

print("Total Spend per Year and Month For All Channels")
snow_df_spend_per_channel.show()

#### 4.3 Gasto total en todos los canales.

Esta transformación, nos permitirá joinear a la tabla de ingresos de modo que tengamos nuestras características de entrada y la variable objetivo en una sola tabla para el entrenamiento del modelo.

In [ ]:
snow_df_spend_per_month = snow_df_spend_per_channel.pivot('CHANNEL',['search_engine','social_media','video','email']).sum('TOTAL_COST').sort('YEAR','MONTH')
snow_df_spend_per_month = snow_df_spend_per_month.select(
    col("YEAR"),
    col("MONTH"),
    col("'search_engine'").as_("SEARCH_ENGINE"),
    col("'social_media'").as_("SOCIAL_MEDIA"),
    col("'video'").as_("VIDEO"),
    col("'email'").as_("EMAIL")
)

print("Total Spend Across All Channels")

# Uncomment the following 'action' statement to execute transformations and view first ten results
snow_df_spend_per_month.show()

#### 4.4 Ingresos totales por año y datos mensuales.

Ahora transformemos los datos de ingresos en ingresos por año/mes usando las funciones `group_by()` y `agg()`.

In [ ]:
snow_df_revenue_per_month = snow_df_revenue.group_by('YEAR','MONTH').agg(sum('REVENUE')).sort('YEAR','MONTH').with_column_renamed('SUM(REVENUE)','REVENUE')

print("Total Revenue per Year and Month")
snow_df_revenue_per_month.show()

#### 4.5 Cruce el gasto total, a los ingresos totales por año y mes en todos los canales.

A continuación, cruzamos estos datos de ingresos con los de gasto de campaña transformados para que nuestras entradas (es decir, costo por canal) y variable objetivo (es decir, ingresos) se puedan cargar en una sola tabla para análisis adicionales y capacitación del modelo.


In [ ]:
snow_df_spend_and_revenue_per_month = snow_df_spend_per_month.join(snow_df_revenue_per_month, ["YEAR","MONTH"])

print("Total Spend and Revenue per Year and Month Across All Channels")
snow_df_spend_and_revenue_per_month.show()

#### 4.6 Examine el Plan de Ejecución de la Consulta.

Snowpark te permite ver la consulta del DataFrame y el plan de ejecución usando la función `explain()`.

In [ ]:
snow_df_spend_and_revenue_per_month.explain()

#### 4.7 Guardar los datos transformados.

Guardaremos los datos transformados en una tabla Snowflake `SPEND_AND_REVENUE_PER_MONTH` para que pueda usarse para análisis adicionales y/o para entrenar un modelo.

In [ ]:
snow_df_spend_and_revenue_per_month.write.mode('overwrite').save_as_table('SNOWPARK_DEMO_DB.SNOWPARK_DEMO_SCHEMA.SPEND_AND_REVENUE_PER_MONTH')

### 5. Usar una UDF para realizar transformaciones.

#### 5.1 Seleccione columnas del DataFrame dfSpendRevenuePerMonthUDF.

Cree un DataFrame a partir de la tabla `SPEND_AND_REVENUE_PER_MONTH` y examinela para descubrir qué información está disponible en este data set.
Seleccione los siguientes campos del DataFrame `dfSpendRevenuePerMonthUDF` usando el método `select()` (recuerde encerrar los nombres de las columnas en una llamada al método `col()`):

- YEAR
- MONTH
- SEARCH_ENGINE
- SOCIAL_MEDIA
- VIDEO
- EMAIL

In [ ]:
tableName = 'SNOWPARK_DEMO_DB.SNOWPARK_DEMO_SCHEMA.SPEND_AND_REVENUE_PER_MONTH'
dfSpendRevenuePerMonthUDF = session.table(tableName)

dfOnTimeReporting = (dfSpendRevenuePerMonthUDF
    .select(
        col("YEAR")
        ,col("MONTH")
        ,col("SEARCH_ENGINE")
        ,col("SOCIAL_MEDIA")
        ,col("VIDEO")
        ,col("EMAIL")
        ))
dfOnTimeReporting.show()

#### 5.2 Usa la UDF para examinar los gastos de inversión más altos.

Invoque la UDF `findTopInvestment` usando el método `call_builtin`, para verificar las inversiones de campaña más altas.
Dar a la UDF los siguientes parámetros:
- SEARCH_ENGINE
- SOCIAL_MEDIA
- VIDEO
- EMAIL


> **&#128221; Note:** Código de la UDL
```
create or replace function SNOWPARK_DEMO_DB.PUBLIC.findTopInvestment(val1 int, text1 text, val2 int, text2 text, val3 int, text3 text, val4 int, text4 text)
returns string
language python
runtime_version = '3.8'
handler = 'findTopInvestment'
as
$$
def findTopInvestment(val1, text1, val2, text2, val3, text3, val4, text4):
    values = {val1: text1, val2: text2, val3: text3, val4: text4}
    max_val = max(values)
    sum_val = sum(values)

    porc_val = max_val / sum_val * 100
    
    return f"{values[max_val]} with {porc_val} of total"
$$;
```
> &#10071; La UDF debe de ser desplegada previamente en tu cuenta.


In [ ]:
dfOnTimeReporting = (dfSpendRevenuePerMonthUDF
    .select(
        col("YEAR")
        ,col("MONTH")
        ,col("SEARCH_ENGINE")
        ,col("SOCIAL_MEDIA")
        ,col("VIDEO")
        ,col("EMAIL")
        ,call_builtin(
            "SNOWPARK_DEMO_DB.PUBLIC.findTopInvestment"
            ,col("SEARCH_ENGINE"), "SEARCH_ENGINE"
            ,col("SOCIAL_MEDIA"), "SOCIAL_MEDIA"
            ,col("VIDEO"), "VIDEO"
            ,col("EMAIL"), "EMAIL").alias("TOP INVESTMENT")
        )
)

    
dfOnTimeReporting.show()

### &#10071; `Shut Down Kernel`
> Después de completar las actividades de esta notebook, apague la misma.